In [1]:
import QAData
from os import listdir
from os.path import isfile, join
import PassageRetrieval as pr
from json2html import *
import matplotlib
#matplotlib.use('Agg')
import pylab as plt
import nlp_utils
import time
import os
import json
import logging
from json_utils import JSONConnector
from QAData import DataSetFactory
import models
from passrtv_models import PassageRetrievalModel
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot
from keras.layers import Convolution1D, Convolution2D
from keras.layers import Input, Embedding, merge, Flatten, SimpleRNN
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Activation, Input, Dense, merge, Dropout, LSTM
from keras.models import Model
from keras.models import Sequential
from keras.layers import Convolution1D, Convolution2D, GlobalMaxPooling2D, Activation, Input, Dense, merge, Dropout
from keras.layers import Flatten, SimpleRNN
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.callbacks import Callback
from itertools import groupby
import sys
import pickle
import nlp_utils
import numpy as np
import threading
import models as models
from models import threadsafe_generator
from scipy import spatial
import nltk
import random

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
params = {
  "expriment_id" : "keras_convnet_$runid",
  "dataset_partitions" : ["train","validate","test"],
  "dataset": "TrecDataSet",
  "ground_truth_file": "/home/aerossom/passage-retrieval/experiments/gold-jakana-test.rank",
  "model" : "KerasConvNetModel_5",
  "modeldes" : "Convolutional NN with 1d input tensor cosine sim input matrix element wise multiplied by salience matrix",
  "out_folder": "/home/aerossom/passage-retrieval/experiments/",
  "working_folder": "/home/aerossom/passage-retrieval/experiments/working_files/",
  "trec_eval_path" : "/home/aerossom/trec_eval.9.0/trec_eval",
  "out_rank_file" : "keras_convnet_matrixsim_$runid.rank",
  "word2vect_path" : "/home/aerossom/datasets/word2vect/GoogleNews-vectors-negative300.bin",
  "preprocess_steps" : [],
  "method_params" : {
                      "optimizer": "rmsprop",
                      "loss":"mean_squared_error",
                      "monitor": "val_loss",
                      "verbose" : 1,
                      "positive_rate": 0.5,
            		      "epochs": 500,
            		      "batch_size": 256,
            		      "validation_size" : 64,
            		      "max_words" : 40,
            		      "patience": 50,
                      "convolution_2d" : {
                        "nb_filter": 64,
                        "nb_row": 3,
                        "nb_col": 3,
                        "subsample": 1,
                        "border_mode" : "valid",
                        "activation" : "relu"
                      },
                      "activation_2nd_Layer" : "relu",
                      "dense_4th_Layer" : 30,
                      "dropout" : 0.1,
                      "dense_6th_layer" : 1,
                      "end_layer_activation" : "sigmoid"
		    }
}

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
w2v_path = '/home/aerossom/datasets/word2vect/GoogleNews-vectors-negative300.bin'
w2v_util = nlp_utils.Word2vectUtils(w2v_path)

In [5]:
#ds = DataSetFactory.loadDataSet('TrecDataSet')
#ds = DataSetFactory.loadDataSet('TrecDataSet_TrainAll')
ds = DataSetFactory.loadDataSet('WikiQADataSet')

qa_pair = {}
qa_pair['train'] = ds.build_qa_pairs(ds.questions['train'])
qa_pair['validate'] = ds.build_qa_pairs(ds.questions['validate'])
qa_pair['test'] = ds.build_qa_pairs(ds.questions['test'])

In [6]:
def avg_precision(y_true, y_pred):
    zipped = zip(y_true, y_pred)
    zipped.sort(key=lambda x:x[1],reverse=True)
    np_y_true, np_y_pred = zip(*zipped)
    k_list = [i for i in range(len(np_y_true)) if int(np_y_true[i])==1]
    score = 0.
    r = np.sum(np_y_true).astype(np.int64)
    for k in k_list:
        Yk = np.sum(np_y_true[:k+1])
        score += Yk/float(k+1)
    if r==0:
        return 0
    score/=(r)
    return score

"""
https://en.wikipedia.org/wiki/Mean_reciprocal_rank
"""
def reciprocal_rank(y_true, y_pred):
    zipped = zip(y_true, y_pred)
    zipped.sort(key=lambda x:x[1],reverse=True)
    count_r = 1.0
    rr_score = 0.0
    for y_t,y_p in zipped:
        if(y_t!=1):
            count_r += 1
        else:
            rr_score = 1.0/count_r
            break
    if count_r-1==len(y_true):
        rr_score = 0.0
    return rr_score

class MAPCallback(Callback):
    
    def __init__(self, validation_data, max_words, proc_funct, filepath, min_delta=0, patience=50, verbose=1, save_best_only=True, save_weights_only=True, period=1):
        super(MAPCallback, self).__init__()
        self.val_ds = validation_data
        self.max_words = max_words
        self.proc_fuction = proc_funct
        self.map_score = []
        self.mrr_score = []
        self.min_delta = min_delta
        #maximize the map
        self.monitor_op = np.greater
        self.patience = patience
        self.period = period
        self.verbose = verbose
        self.save_best_only = save_best_only
        self.save_weights_only = save_weights_only
        self.epochs_since_last_save = 0
        self.filepath = filepath
        self.min_delta *= -1
        self.stopped_epoch = 0
    
    def on_train_begin(self, logs=None):
        self.wait = 0  # Allow instances to be re-used
        self.best = np.Inf if self.monitor_op == np.less else -np.Inf

    def on_epoch_end(self, epoch, logs={}):
        current_map, current_mrr = self.calculate_map_mrr()
        self.save_model(epoch, logs, current_map)
        logging.info("MAP evaluation - epoch: {:d} - score: {:.6f}".format(epoch, current_map))
        logging.info("MRR evaluation - epoch: {:d} - score: {:.6f}".format(epoch, current_mrr))
        if current_map is None:
            warnings.warn('MAP Early stopping requires %s available!' %
                          (self.monitor), RuntimeWarning)

        if self.monitor_op(current_map - self.min_delta, self.best):
            self.best = current_map
            self.wait = 0
        else:
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
            self.wait += 1

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0 and self.verbose > 0:
        #    print('Epoch %05d: MAP early stopping' % (self.stopped_epoch))
            logging.info("MAP early stopping Epoch {:d} evaluation - MAP: {:.6f} ".format(self.stopped_epoch,self.best))
    
    def save_model(self, epoch, logs, cmap):
        logs = logs or {}
        self.epochs_since_last_save += 1
        if self.epochs_since_last_save >= self.period:
            self.epochs_since_last_save = 0
            filepath = self.filepath.format(epoch=epoch, **logs)
            if self.save_best_only:
                current = cmap
                if current is None:
                    warnings.warn('Can save best model only with %s available, '
                                  'skipping.' % (' MAP '), RuntimeWarning)
                else:
                    if self.monitor_op(current, self.best):
                        if self.verbose > 0:
                            print('Epoch %05d: %s improved from %0.5f to %0.5f,'
                                  ' saving model to %s'
                                  % (epoch, ' MAP ', self.best,
                                     current, filepath))
                        self.best = current
                        if self.save_weights_only:
                            self.model.save_weights(filepath, overwrite=True)
                        else:
                            self.model.save(filepath, overwrite=True)
                    else:
                        if self.verbose > 0:
                            print('Epoch %05d: %s did not improve' %
                                  (epoch, ' MAP '))
            else:
                if self.verbose > 0:
                    print('Epoch %05d: saving model to %s' % (epoch, filepath))
                if self.save_weights_only:
                    self.model.save_weights(filepath, overwrite=True)
                else:
                    self.model.save(filepath, overwrite=True)

    def calculate_map_mrr(self):
        y_true = []
        y_pred = []
        avg_p = []
        rr_list = []
        num_groups = 0
        for key, group in groupby(self.val_ds, lambda x: x.qi):
            samples = []
            is_any_true = False
            for q_g in group:
                samples.append(q_g)
                if q_g.l == 1:
                    is_any_true = True
            #Just take the 1/3 of the dataset randomly
            if random.uniform(0, 10) > 8:
                is_any_true = False
            if is_any_true:
                x, y_true = self.proc_fuction(samples,max_terms=self.max_words)
                y_pred = self.model.predict_proba(x, verbose=0)
                avg_p_score = avg_precision(y_true, y_pred)
                avg_p.append( avg_p_score )
                rr_score = reciprocal_rank(y_true, y_pred)
                rr_list.append( rr_score )
                #print y_true
                #print y_pred
                #print 'avg_prec = ', avg_p_score, ', rr = ', rr_score
                num_groups += 1
        cmap_score = sum(avg_p)/num_groups
        cmrr_score = sum(rr_list)/num_groups
        self.map_score.append(cmap_score)
        self.mrr_score.append(cmrr_score)
        #print 'map = ', cmap_score, ', mrr = ', cmrr_score
        return cmap_score, cmrr_score

In [39]:
"""
Convolutional NN with 1d input tensor cosine sim input matrix element wise multiplied by salience matrix
"""
class KerasConvNetModel_5(models.PassageRetrievalModel):

    def __init__(self,init_params):
        super(KerasConvNetModel_5, self).__init__('KerasConvNetModel_5', init_params['runid'])
        self.w2vutil = init_params['w2v']
        self.w2v = self.w2vutil.getWord2VectModel()
        self.params = init_params['params']
        self.max_words = self.params['method_params']['max_words']
        self.positive_rate = self.params['method_params']['positive_rate']
        self.prep_step = [ str(step) for step in init_params['params']['preprocess_steps'] ]

    def getSalienceScore(self, qv, av, maxterms=40):
        score = 0
        #print qv, av
        imp_postag = set(['WRB','VB', 'VBD', 'VBG', 'VBN', 'VBP','VBZ', 'WDT', 'WP', 'WRB', 'NN', 'NNS', 'NNP', 'NNPS', 'MD'])
        pw1 = nltk.pos_tag(qv)
        pw2 = nltk.pos_tag(av)
        if len(pw1)>maxterms:
            pw1 = pw1[0:maxterms]
        if len(pw2)>maxterms:
            pw2 = pw2[0:maxterms]
        w1_m = np.zeros((maxterms,maxterms))
        w2_m = np.zeros((maxterms,maxterms))
        pw1_l = [len(set([w1[1]]).intersection(imp_postag)) for w1 in pw1]
        w1_m[:,0:len(pw1_l)]=pw1_l
        pw2_l = [len(set([w2[1]]).intersection(imp_postag)) for w2 in pw2]
        w2_m[:,0:len(pw2_l)]=pw2_l
        out_m = (w1_m + w2_m.T)/2
        #print pw1_l, pw2_l, pw1
        return out_m[0:maxterms,0:maxterms]

    def buildCosineSimMatrix(self, questions_answer_pairs, max_terms=20):
        #Construct Question Answer Matrix Pairs
        x = []
        y = []
        for pair in questions_answer_pairs:
            q_list = nlp_utils.data_preprocess(pair.q,self.prep_step)
            a_list = []
            q_vect = self.w2vutil.transform2Word2Vect(q_list)
            cos_matrix = []
            sal_matrix = []
            for i, q_i in enumerate(q_vect):
                if i==max_terms:
                    break
                sim_qi_a = []
                a_list = nlp_utils.data_preprocess(pair.a,self.prep_step)
                a_vect = self.w2vutil.transform2Word2Vect(a_list)
                for k, a_k in enumerate(a_vect):
                    if k==max_terms:
                        break
                    sim_qi_a += [spatial.distance.cosine(q_i, a_k)]
                cos_matrix += [sim_qi_a]
            sal_matrix = self.getSalienceScore(q_list,a_list,max_terms)
            cos_matrix = np.array(cos_matrix)
            #sal_matrix = np.array(sal_matrix)
            shape_cos_matrix = cos_matrix.shape
            #print 'shapes: ', sal_matrix.shape, cos_matrix.shape
            cos_matrix = np.pad(cos_matrix, ((0,max_terms-shape_cos_matrix[0]),(0,max_terms-shape_cos_matrix[1])), \
                                mode='constant', constant_values=2)
            #sal_matrix = np.pad(sal_matrix, ((0,max_terms-shape_cos_matrix[0]),(0,max_terms-shape_cos_matrix[1])), mode='constant')
            if np.isnan(cos_matrix).any():
                print 'ERROR IS NAN: ',pair
            #x.append( np.expand_dims(np.multiply(cos_matrix, sal_matrix),0) )
            x.append( np.expand_dims(cos_matrix,0) )
            y.append( pair.l )
        return np.array(x), np.array(y)

    def load_model(self):
        self.model = Sequential()
        mp = self.params['method_params']
        self.model.add(Convolution2D(
            nb_filter=mp['convolution_2d']['nb_filter'],
            nb_row=mp['convolution_2d']['nb_row'],
            nb_col=mp['convolution_2d']['nb_col'],
            subsample=(mp['convolution_2d']['subsample'], mp['convolution_2d']['subsample']),
            border_mode=mp['convolution_2d']['border_mode'],
            activation=mp['convolution_2d']['activation'],
            input_shape=(1, self.max_words, self.max_words)))
        self.model.add(Activation(mp['activation_2nd_Layer']))
        self.model.add(GlobalMaxPooling2D())
        self.model.add(Dense(mp['dense_4th_Layer']))
        self.model.add(Dropout(mp['dropout']))
        self.model.add(Dense(mp['dense_6th_layer']))
        self.model.add(Activation(mp['end_layer_activation']))
        return self.model

    @threadsafe_generator
    def generateXYBatches(self, ds, dataset, num_samples, positive_rate=0.5):
        samples = ds.get_random_samples(dataset, num_samples, self.positive_rate)
        while 1:
            x, y = self.buildCosineSimMatrix(samples,max_terms=self.max_words)
            yield ( x, y )

    def train(self, ds, qa_pair):
        self.model = self.load_model()
        self.best_params=self.params['working_folder']+self.params['expriment_id'].replace('$runid',self.runid)+"_best.hdf5"
        #if want to load the best weights in other training
        #self.model.load_weights(self.best_params)
        #MAP CallBack
        map_callback = MAPCallback(qa_pair['validate'], self.max_words, self.buildCosineSimMatrix, self.best_params)
        # checkpoints
        #output the model weights each time an improvement is observed during training
        #checkpoint = ModelCheckpoint(self.best_params, monitor=self.params['method_params']['monitor'], verbose=self.params['method_params']['verbose'], save_best_only=True, mode='auto')
        #stops if the model is not learning at any point
        #earlyStopping= EarlyStopping(monitor=self.params['method_params']['monitor'], patience=self.params['method_params']['patience'], verbose=self.params['method_params']['verbose'], mode='auto')
        
        epochs_number = self.params['method_params']['epochs']
        batch_size = self.params['method_params']['batch_size']
        validation_size = self.params['method_params']['validation_size']

        self.model.compile(loss='mean_squared_error',
                      optimizer='rmsprop',
                      metrics=['accuracy'])

        history = self.model.fit_generator(
                    self.generateXYBatches(ds, qa_pair['train'], batch_size, positive_rate=self.positive_rate),
                    samples_per_epoch=batch_size,
                    validation_data=self.generateXYBatches(ds, qa_pair['validate'], validation_size, positive_rate=self.positive_rate),
                    nb_val_samples=validation_size,
                    nb_epoch=epochs_number,
                    callbacks=[map_callback]
                    #callbacks=[checkpoint, earlyStopping, map_callback]
                    )
        
        #Add MAP and MRR to history
        history.history['map'] = map_callback.map_score
        history.history['mrr'] = map_callback.mrr_score
        #Save history object in pickel
        self.save_history( self.best_params.replace('_best.hdf5','_history.pkl'), history )
        return history
    
    def save_history(self,out_file, history):
        with open(out_file, 'wb') as output:
            pickle.dump(history.history, output, pickle.HIGHEST_PROTOCOL)
        """
        To Open 
            with open('in_file.pkl', 'rb') as input:
                history = pickle.load(input)
        """

    def test(self, ds, qa_pairs):
        #reload best weights
        self.model.load_weights(self.best_params)
        #Construct Test dataset
        test_qxa, test_l = self.buildCosineSimMatrix(qa_pairs, max_terms=self.max_words)
        predictions = self.model.predict(np.array(test_qxa))
        return predictions

In [8]:
in_file = '/home/aerossom/passage-retrieval/experiments/working_files/keras_convnet_test_run_history.pkl'

In [19]:
with open(in_file, 'rb') as input:
    history = pickle.load(input)

In [9]:
model_params = {"w2v": w2v_util, "runid":'test_run',  "params":params }
k_model = KerasConvNetModel_5(model_params)
#history = k_model.train(ds, qa_pair)

In [35]:
#First predictions
k_model = KerasConvNetModel_5(model_params)
test_qxa, test_l = k_model.buildCosineSimMatrix(qa_pair['test'], max_terms=40)
train_qxa, train_l = k_model.buildCosineSimMatrix(qa_pair['train'], max_terms=40)

In [36]:
print qa_pair['train'][0].q, qa_pair['train'][0].a

how are glacier caves formed ? A partly submerged glacier cave on Perito Moreno Glacier .


In [38]:
np.set_printoptions(precision=1)
np.set_printoptions(threshold='nan')
np.set_printoptions(suppress=True)
#print test_qxa[0]
print np.array_str(train_qxa[0], precision=1, suppress_small=True)

[[[ 1.   0.9  0.9  1.   0.9  0.8  1.1  0.9  1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1. ]
  [ 0.9  0.9  1.   1.   0.9  0.9  1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1. ]
  [ 1.   0.9  0.7  0.   0.6  1.   1.   0.9  0.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1. ]
  [ 1.   0.9  0.8  0.6  0.2  1.   1.   0.9  0.6  1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1. ]
  [ 1.   0.9  0.9  1.   0.9  1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.   1.
    1.   1.   

In [30]:
spatial.distance.cosine(w2v_util.w2v_model['glacier'], w2v_util.w2v_model['oil'])

0.87770495563745499

In [70]:
from keras import models
model = models.load_model('/home/aerossom/passage-retrieval/experiments/working_files/keras_convnet_1492099016_best.hdf5')

th


In [71]:
from nlp_utils import avg_precision
from nlp_utils import reciprocal_rank
#Make the first preditions
first_predictions = model.predict(np.array(test_qxa))
rank_file_1 = 'experiments/rankfile_0.rank'
idx_pred = 0
rank_text = ''
for key,group in groupby(qa_pair['test'], lambda x: x.qi):
    for i, qp in enumerate(group):
        rank_text += str(qp.qi) + ' 0 ' + str(qp.ai) + ' 0 ' + str(first_predictions[idx_pred][0]) + ' 0\n'
        idx_pred += 1
with open(rank_file_1, 'wb') as text_file:
    text_file.write(rank_text)

In [73]:
!/home/aerossom/trec_eval.9.0/trec_eval -c /home/aerossom/datasets/WikiQACorpus/WikiQA-test-filtered.ref \
experiments/rankfile_0.rank

runid                 	all	0
num_q                 	all	237
num_ret               	all	2341
num_rel               	all	283
num_rel_ret           	all	283
map                   	all	0.5940
gm_map                	all	0.4601
Rprec                 	all	0.4339
bpref                 	all	0.4259
recip_rank            	all	0.6047
iprec_at_recall_0.00  	all	0.6096
iprec_at_recall_0.10  	all	0.6096
iprec_at_recall_0.20  	all	0.6096
iprec_at_recall_0.30  	all	0.6035
iprec_at_recall_0.40  	all	0.6014
iprec_at_recall_0.50  	all	0.6014
iprec_at_recall_0.60  	all	0.5902
iprec_at_recall_0.70  	all	0.5902
iprec_at_recall_0.80  	all	0.5863
iprec_at_recall_0.90  	all	0.5863
iprec_at_recall_1.00  	all	0.5863
P_5                   	all	0.1890
P_10                  	all	0.1118
P_15                  	all	0.0785
P_20                  	all	0.0595
P_30                  	all	0.0398
P_100                 	all	0.0119
P_200                 	all	0.0060
P_500                 	all	0.0024
P_1000                	all	0.0

In [72]:
!/home/aerossom/trec_eval.9.0/trec_eval -c /home/aerossom/passage-retrieval/experiments/gold-jakana-test2.rank \
experiments/rankfile_0.rank

trec_eval: No queries with both results and relevance info


In [74]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [ ]:
from QAData import QAPair

first_map = []
second_map = []
final_1_predictions = []
final_2_predictions = []
ids_q = []
for key,group in groupby(qa_pair['test'], lambda x: x.qi):
    first_qapair = []
    for qp in group:
        first_qapair.append(qp)
    # Get first predictions without rerank
    first_qxa, first_l = k_model.buildCosineSimMatrix(first_qapair, max_terms=k_model.max_words)
    first_predictions = model.predict(np.array(first_qxa))
    first_map.append(avg_precision(first_l, first_predictions))
    final_1_predictions.extend(first_predictions)
    # Get second preditions taking into account the pseudo relevance feedback
    maxq = np.argmax(first_predictions)
    #q_1 = ' '.join([word for word in first_qapair[maxq].q.split() if word not in (stopwords.words('english'))])
    a_1 = ' '.join([word for word in first_qapair[maxq].a.split() if word not in (stopwords.words('english'))])
    q = ' '.join([word for word in qp.q.split() if word not in (stopwords.words('english'))])
    second_qapair = [ QAPair(qp.qi, q+' '+a_1,qp.ai,qp.a,qp.l) for qp in first_qapair ]
    second_qxa, second_l = k_model.buildCosineSimMatrix(second_qapair, max_terms=k_model.max_words)
    second_predictions = model.predict(np.array(second_qxa))
    second_predictions[maxq] = 1.0
    second_map.append(avg_precision(second_l, second_predictions))
    final_2_predictions.extend(second_predictions)
    print first_qapair[0].qi, ' l=',sum(first_l), 'map_1: ', first_map[len(first_map)-1] ,' map_2: ', second_map[len(second_map)-1]
    #print first_map, second_map

1  l= 1 map_1:  1.0  map_2:  1.0
2  l= 0 map_1:  0  map_2:  0
3  l= 1 map_1:  0.25  map_2:  0.5
4  l= 1 map_1:  1.0  map_2:  1.0
5  l= 4 map_1:  0.15499465812  map_2:  0.326923076923
6  l= 0 map_1:  0  map_2:  0
7  l= 0 map_1:  0  map_2:  0
8  l= 0 map_1:  0  map_2:  0
9  l= 0 map_1:  0  map_2:  0
10  l= 1 map_1:  0.333333333333  map_2:  0.25
11  l= 1 map_1:  1.0  map_2:  1.0
12  l= 0 map_1:  0  map_2:  0
13  l= 0 map_1:  0  map_2:  0
14  l= 0 map_1:  0  map_2:  0
15  l= 0 map_1:  0  map_2:  0
16  l= 0 map_1:  0  map_2:  0
17  l= 0 map_1:  0  map_2:  0
18  l= 0 map_1:  0  map_2:  0
19  l= 0 map_1:  0  map_2:  0
20  l= 2 map_1:  0.7  map_2:  0.833333333333
21  l= 1 map_1:  1.0  map_2:  1.0
22  l= 1 map_1:  0.25  map_2:  0.111111111111
23  l= 0 map_1:  0  map_2:  0
24  l= 1 map_1:  0.5  map_2:  0.1
25  l= 0 map_1:  0  map_2:  0
26  l= 0 map_1:  0  map_2:  0
27  l= 2 map_1:  0.22619047619  map_2:  0.22619047619
28  l= 1 map_1:  1.0  map_2:  1.0
29  l= 1 map_1:  0.5  map_2:  0.2
30  l= 2 m

In [57]:
print sum(first_map)/len(first_map)

0.644079734044


In [58]:
print sum(second_map)/len(second_map)

0.669580894184


In [59]:
rank_file_1 = 'experiments/rankfile_1.rank'
idx_pred = 0
rank_text = ''
for key,group in groupby(qa_pair['test'], lambda x: x.qi):
    for i, qp in enumerate(group):
        rank_text += str(qp.qi) + ' 0 ' + str(qp.ai) + ' 0 ' + str(final_2_predictions[idx_pred][0]) + ' 0\n'
        idx_pred += 1
with open(rank_file_1, 'wb') as text_file:
    text_file.write(rank_text)

In [24]:
!/home/aerossom/trec_eval.9.0/trec_eval -c /home/aerossom/datasets/WikiQACorpus/WikiQA-test-filtered.ref \
experiments/rankfile_1.rank

trec_eval: No queries with both results and relevance info


In [62]:
!/home/aerossom/trec_eval.9.0/trec_eval -c /home/aerossom/passage-retrieval/experiments/gold-jakana-test.rank experiments/rankfile_1.rank

runid                 	all	0
num_q                 	all	95
num_ret               	all	1517
num_rel               	all	284
num_rel_ret           	all	284
map                   	all	0.6696
gm_map                	all	0.3002
Rprec                 	all	0.5847
bpref                 	all	0.5833
recip_rank            	all	0.7004
iprec_at_recall_0.00  	all	0.7276
iprec_at_recall_0.10  	all	0.7276
iprec_at_recall_0.20  	all	0.7136
iprec_at_recall_0.30  	all	0.7091
iprec_at_recall_0.40  	all	0.6982
iprec_at_recall_0.50  	all	0.6932
iprec_at_recall_0.60  	all	0.6824
iprec_at_recall_0.70  	all	0.6748
iprec_at_recall_0.80  	all	0.6380
iprec_at_recall_0.90  	all	0.6339
iprec_at_recall_1.00  	all	0.6309
P_5                   	all	0.3684
P_10                  	all	0.2411
P_15                  	all	0.1768
P_20                  	all	0.1358
P_30                  	all	0.0958
P_100                 	all	0.0299
P_200                 	all	0.0149
P_500                 	all	0.0060
P_1000                	all	0.00

In [ ]:
history

In [20]:
%matplotlib inline
# Print learning history
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

AttributeError: 'dict' object has no attribute 'history'

In [17]:
history.history

{'MAP': [0.50040411122804629, 0.52189223593768197, 0.55762187871781732],
 'MRR': [0.5037535065417272, 0.5279446569060693, 0.5645427026029268],
 'acc': [0.4765625, 0.4921875, 0.54296875],
 'loss': [0.25362694263458252, 0.25330552458763123, 0.24734991788864136],
 'val_acc': [0.5, 0.515625, 0.484375],
 'val_loss': [0.25115877389907837, 0.24868617951869965, 0.24784004688262939]}

In [23]:
with open('out_file', 'wb') as output:
    pickle.dump(history.history, output, pickle.HIGHEST_PROTOCOL)